In [2]:
import mcpi.minecraft as minecraft
import mcpi.block as block
import mcpi.entity as entity
from mcpi.vec3 import Vec3
import numpy as np
import random
import pickle
import glob
import time
import os
from PIL import Image
import pytmx
import os

NORTH = 'z0'
EAST = 'x1'
SOUTH = 'z1'
WEST = 'x0'

TORCH_DIR_X1 = 1
TORCH_DIR_X0 = 2
TORCH_DIR_Z1 = 3
TORCH_DIR_Z0 = 4
TORCH_DIR_UP = 5

TILE_SIZE = 9

ATTACHABLES = [
    block.TORCH[0],
    block.REDSTONE_TORCH_INACTIVE[0],
    block.REDSTONE_TORCH_ACTIVE[0],
    block.DOOR_ACACIA[0],
    block.DOOR_BIRCH[0],
    # Fixed in the blocks.py to 197:
    block.DOOR_DARK_OAK[0],
    block.DOOR_IRON[0],
    block.DOOR_JUNGLE[0],
    block.DOOR_SPRUCE[0],
    block.DOOR_WOOD[0],
    (block.SIGN())[0],
    block.CARPET[0],
]

FACING = [
    block.TORCH[0],
    block.REDSTONE_TORCH_INACTIVE[0],
    block.REDSTONE_TORCH_ACTIVE[0],
]

# TODO implement rotation
ROT = [
    block.SKULL[0],
]

def to_tile(x, y, z):
    return [int(x/TILE_SIZE), int(y/TILE_SIZE), int(z/TILE_SIZE)]

def to_world(x, y, z):
    return [int(x*TILE_SIZE), int(y*TILE_SIZE), int(z*TILE_SIZE)]

def to_rot(sx, sy, sz, x, y, z, deg):
    if deg==0:
        return [sx+x, sy+y, sz+z]
    if deg==90:
        return [sx+TILE_SIZE-1-z, sy+y, sz+x]
    if deg==180:
        return [sx+TILE_SIZE-1-x, sy+y, sz+TILE_SIZE-1-z]
    if deg==270:
        return [sx+z, sy+y, sz+TILE_SIZE-1-x]    

def to_torch_dir_rot(dir, deg):
    progression = {
        TORCH_DIR_X1: TORCH_DIR_Z1,
        TORCH_DIR_Z1: TORCH_DIR_X0,
        TORCH_DIR_X0: TORCH_DIR_Z0,
        TORCH_DIR_Z0: TORCH_DIR_X1,
    }
    if deg==0:
        return dir
    if deg==90:
        return progression[dir]
    if deg==180:
        return progression[progression[dir]]
    if deg==270:
        return progression[progression[progression[dir]]]

In [2]:
def load_room(path):
    with open(path, 'rb') as f:
        spec = pickle.load(f)
        return Room(
            name=spec['name'],
            side=spec['side'],
            door_style=spec['door_style'],
            source_location=spec['source_location'],
            version=spec['version'],
            blocks=spec['blocks'],
        )

class Room:
    def __init__(self, name, side, door_style, source_location, version, blocks):
        self.name = name
        self.side = side
        self.door_style = door_style
        self.source_location = source_location
        self.version = version
        self.blocks = blocks

    def draw(self, px, py, pz, rot=0):
        px = float(px)
        py = float(py)
        pz = float(pz)

        i = 0
        for y in np.arange(0, self.side, 1):
            for x in np.arange(0, self.side, 1):
                for z in np.arange(0, self.side, 1):
                    b = self.blocks[i]
                    if b[0] not in ATTACHABLES:
                        mc.setBlockWithNBT(*to_rot(px, py, pz, x, y, z, rot), b)
                    i += 1
        
        i = 0
        for y in np.arange(0, self.side, 1):
            for x in np.arange(0, self.side, 1):
                for z in np.arange(0, self.side, 1):
                    b = self.blocks[i]
                    if b[0] in FACING:
                        br = block.Block(b[0], to_torch_dir_rot(b[1], rot), b[2])
                        mc.setBlockWithNBT(*to_rot(px, py, pz, x, y, z, rot), br)
                    elif b[0] in ATTACHABLES:
                        mc.setBlockWithNBT(*to_rot(px, py, pz, x, y, z, rot), b)
                    i += 1

    def get_block_at(self,x,y,z):
        return self.blocks[y*self.side*self.side+x*self.side+z]

    def door_dir(self):
        # X1 = tile image down
        # X0 = tile image up
        # Z1 = tile image right
        # Z0 = tile image left
        conversion = {
            'down': 'x1',
            'up': 'x0',
            'right': 'z1',
            'left': 'z0',
        }

        return conversion[self.door_style]
    
    def get_image_at_y(self, y=1):
        pixels = []
        for z in np.arange(0, self.side, 1):
            for x in np.arange(0, self.side, 1):
                #px = self.get_block_at(*to_rot(0,0,0,x,y,z,180)).getRGBA()
                px = self.get_block_at(x,y,z).getRGBA()
                pixels.append([px[0], px[1], px[2]])

        pixels = np.array(pixels, dtype=np.uint8).reshape((9,9,3))
        return Image.fromarray(pixels)

    def write(self, dir='rooms'):
        with open('%s/%s.pickle' % (dir, r), 'wb') as f:
            pickle.dump({
                'name': self.name,
                'side': self.side,
                'door_style': self.door_style,
                'source_location': self.source_location,
                'version': self.version,
                'blocks': self.blocks,
            }, f)

63

In [7]:
#mc = minecraft.Minecraft.create(address='192.168.1.38')
mc = minecraft.Minecraft.create(address='192.168.1.100')
mc.player.getPos()

Vec3(1.5,233.0,1.5)

In [28]:
side = 9
p = mc.player.getPos()
p.y = p.y-1
for x in np.arange(0, side, 1):
    for y in np.arange(0, side, 1):
        for z in np.arange(0, side, 1):
            if x==0 or x==side-1 or y==0 or y==side-1 or z==0 or z==side-1:
                mc.setBlock(p.x+x, p.y+y, p.z+z, block.STONE)
                


In [18]:
#print(p)
coords = {}
coords['bathroom'] = Vec3(171,0,158)
coords['dining'] = Vec3(319,-1,137)
coords['library'] = Vec3(336, -1, 96)
coords['factory'] = Vec3(366, -1, 75)
coords['clothes_shop'] = Vec3(399, -1, 48)
coords['swimming_pool'] = Vec3(377, -1, -33)
coords['school'] = Vec3(364, -1, -113)
coords['lazerforce'] = Vec3(359, -1, -101)
coords['train_station'] = Vec3(341, -1, -64)
coords['castle'] = Vec3(260, -1, -92)
coords['spider_room'] = Vec3(240, -1, -50)
coords['sticker_room'] = Vec3(287, -1, -37)
coords['castle_dungeon'] = Vec3(267, -1, -1)
coords['war_room'] = Vec3(414, -1, -73)
coords['bank'] = Vec3(439, -1, -121)
coords['pokemon_room'] = Vec3(454, -1, -84)
coords['dice_room'] = Vec3(415, -1, -105)
coords['miners_cave'] = Vec3(387, -1, -74)

normalise = {
    'down': 270,
    'left': 0,
    'up': 90,
    'right': 180,
}

doordir = {}
doordir['bathroom'] = 'down'
doordir['dining'] = 'left'
doordir['library'] = 'right'
doordir['factory'] = 'left'
doordir['clothes_shop'] = 'left'
doordir['swimming_pool'] = 'left'
doordir['school'] = 'up'
doordir['lazerforce'] = 'up'
doordir['train_station'] = 'up'
doordir['castle'] = 'up'
doordir['spider_room'] = 'left'
doordir['sticker_room'] = 'right'
doordir['castle_dungeon'] = 'left'
doordir['war_room'] = 'down'
doordir['bank'] = 'up'
doordir['pokemon_room'] = 'down'
doordir['dice_room'] = 'left'
doordir['miners_cave'] = 'up'

In [25]:
for r in coords.keys():
    print('Processing %s...' % r)
    side = 9
    blocks = mc.getBlocksWithNBT(coords[r].x, coords[r].y, coords[r].z, coords[r].x+side-1, coords[r].y+side-1, coords[r].z+side-1)
    room_obj = Room(
        name=r,
        side=9,
        door_style=doordir[r],
        source_location=coords[r],
        version=1,
        blocks=blocks,
    )
    room_obj.write(dir='rooms')



Processing bathroom...
Processing dining...
Processing library...
Processing factory...
Processing clothes_shop...
Processing swimming_pool...
Processing school...
Processing lazerforce...
Processing train_station...
Processing castle...
Processing spider_room...
Processing sticker_room...
Processing castle_dungeon...
Processing war_room...
Processing bank...
Processing pokemon_room...
Processing dice_room...
Processing miners_cave...


In [8]:
rooms = {}
for rf in glob.glob("rooms/*.pickle"):
    room = load_room(rf)
    rooms[room.name] = room
    


#rooms['library'].draw(*to_world(0,0,7), rot=180)

In [57]:
for r in rooms.keys():
    rooms[r].get_image_at_y().save('rooms_images/%s.png' % r)

In [5]:
rooms['dining'].door_dir()

'z0'

In [147]:
cp = mc.player.getPos()
mc.player.setPos(-1, 0, -1)


In [46]:
mc.player.setPos(cp)

In [129]:
i = 0
for y in np.arange(0, side, 1):
    for x in np.arange(0, side, 1):
        for z in np.arange(0, side, 1):
            mc.setBlockWithNBT(0.0+x, 0.0+y, 0.0+z, rooms['factory']['blocks'][i])
            i += 1

In [9]:
def draw_corridor(cx,cy,cz,rot=0,holes=[]):
    cx = float(cx)
    cy = float(cy)
    cz = float(cz)
    
    side = 9
    is_hole = {
        'x0': lambda x,y,z: x==0 and y!=0 and y!=side-1 and z!=0 and z!=side-1,
        'x1': lambda x,y,z: x==side-1 and y!=0 and y!=side-1 and z!=0 and z!=side-1,
        'y0': lambda x,y,z: y==0 and x!=0 and x!=side-1 and z!=0 and z!=side-1,
        'y1': lambda x,y,z: y==side-1 and x!=0 and x!=side-1 and z!=0 and z!=side-1,
        'z0': lambda x,y,z: z==0 and y!=0 and y!=side-1 and x!=0 and x!=side-1,
        'z1': lambda x,y,z: z==side-1 and y!=0 and y!=side-1 and x!=0 and x!=side-1,
    }

    for x in np.arange(0, side, 1):
        for y in np.arange(0, side, 1):
            for z in np.arange(0, side, 1):
                mc.setBlock(cx+x, cy+y, cz+z, block.AIR)

    for x in np.arange(0, side, 1):
        for y in np.arange(0, side, 1):
            for z in np.arange(0, side, 1):
                wall = False
                if x==0 or x==side-1 or y==0 or y==side-1 or z==0 or z==side-1:
                    wall = True
                for hole in holes:
                    if is_hole[hole](x,y,z):
                        wall = False
                if wall:
                    mc.setBlock(*to_rot(cx, cy, cz, x, y, z, rot), block.STONE_BRICK)
                else:
                    if y==side-2 and random.randint(1, 4)==1:
                        mc.setBlock(*to_rot(cx, cy, cz, x, y, z, rot), block.COBWEB)
                    else:
                        mc.setBlock(*to_rot(cx, cy, cz, x, y, z, rot), block.AIR)

    torch_wall = 'y0'
    torch_wall = set(set(is_hole.keys()).difference(set(holes))).difference(['y1', 'y0'])
    if len(torch_wall)>0:
        torch_wall = torch_wall.pop()
    halfside = int(side/2)
    if torch_wall=='x0':
        mc.setBlock(*to_rot(cx, cy, cz, 1, halfside, halfside, rot), block.TORCH[0], to_torch_dir_rot(TORCH_DIR_X1, rot))
    elif torch_wall=='x1':
        mc.setBlock(*to_rot(cx, cy, cz, side-2, halfside, halfside, rot), block.TORCH[0], to_torch_dir_rot(TORCH_DIR_X0, rot))
    elif torch_wall=='z0':
        mc.setBlock(*to_rot(cx, cy, cz, halfside, halfside, 1, rot), block.TORCH[0], to_torch_dir_rot(TORCH_DIR_Z1, rot))
    elif torch_wall=='z1':
        mc.setBlock(*to_rot(cx, cy, cz, halfside, halfside, side-2, rot), block.TORCH[0], to_torch_dir_rot(TORCH_DIR_Z0, rot))
    else:
        mc.setBlock(*to_rot(cx, cy, cz, halfside, 1, halfside, rot), block.STONE_BRICK_CHISELED)
        mc.setBlock(*to_rot(cx, cy, cz, halfside, 2, halfside, rot), block.TORCH[0], TORCH_DIR_UP)

#draw_corridor(*to_world(0,0,7), rot=0, holes=['z0', 'x0', 'x1', 'z1'])


In [52]:
# Generate corridor tile images
corridor_types = {
    'deadend': ['x0'],
    'corner': ['x0', 'z0'],
    'straight': ['x0', 'x1'],
    'tee': ['x0', 'x1', 'z0'],
    'intersection': ['x0', 'x1', 'z0', 'z1'],
}

for c in corridor_types.keys():
    draw_corridor(*to_world(0,0,5), corridor_types[c])
    pos = to_world(0,0,5)
    blocks = mc.getBlocksWithNBT(pos[0], pos[1], pos[2], pos[0]+TILE_SIZE-1, pos[1]+TILE_SIZE-1, pos[2]+TILE_SIZE-1)
    room_obj = Room(
        name=c,
        side=9,
        door_style='corridor',
        source_location=coords[r],
        version=1,
        blocks=blocks,
    )
    i = room_obj.get_image_at_y().save('rooms_images/%s.png' % c)

z0
x1
z0
z1
set()


In [79]:
mc.player.setPos(0,0,0)

In [31]:
mc = minecraft.Minecraft.create(address='192.168.1.38')

In [29]:
#draw_corridor(*to_world(5,0,0), ['x1', 'x0', 'z1'])
#draw_corridor(*to_world(1,0,0), ['x0', 'x1'])
#rooms['bathroom'].draw(*to_world(-1,0,0))

#draw_corridor(*to_world(0,0,5), ['x1', 'x0', 'z1'])

tmxdata = pytmx.TiledMap("m.tmx")
for layer in tmxdata.visible_layers:
    if layer.name=='dungeon':
        dungeon = layer
    if layer.name=='mobs':
        mobs = layer


In [21]:
for t in dungeon:
    if t.type!=None:
        type = t.type
    else:
        type = os.path.splitext(os.path.basename(t.image[0]))[0]

    if type not in rooms.keys() and type not in ['tee', 'straight', 'intersection', 'corner', 'deadend']:
        continue
    if t.rotation not in [0,90,180,270]:
        continue

    # Force origin to the same spot, we are rotating around a centre
    x = int(t.x/TILE_SIZE)
    z = int(t.y/TILE_SIZE)
    if t.rotation==90 or t.rotation==180:
        z = z+1
    if t.rotation==180 or t.rotation==270:
        x = x-1

    if type in rooms:
        rooms[type].draw(*to_world(x,0,z), rot=t.rotation)
    elif type=='tee':
        draw_corridor(*to_world(x,0,z), rot=t.rotation, holes=[NORTH, WEST, SOUTH])
    elif type=='straight':
        draw_corridor(*to_world(x,0,z), rot=t.rotation, holes=[NORTH, SOUTH])
    elif type=='intersection':
        draw_corridor(*to_world(x,0,z), rot=t.rotation, holes=[NORTH, EAST, SOUTH, WEST])
    elif type=='corner':
        draw_corridor(*to_world(x,0,z), rot=t.rotation, holes=[NORTH, WEST])
    elif type=='deadend':
        draw_corridor(*to_world(x,0,z), rot=t.rotation, holes=[NORTH])

In [32]:
for t in mobs:
    if t.type!=None:
        type = t.type
    else:
        type = os.path.splitext(os.path.basename(t.image[0]))[0]

    x = int(t.x)
    z = int(t.y)

    print(x,z,type)
    if type=='zombie':
        mc.spawnEntity(entity.ZOMBIE, x, 1, z)
    elif type=='skeleton':
        mc.spawnEntity(entity.SKELETON, x, 1, z, '{HandItems:[{id:beetroot,Count:1},{id:bone,Count:1}]}')
    elif type=='enderman':
        mc.spawnEntity(entity.ENDERMAN, x, 1, z)
    elif type=='creeper':
        mc.spawnEntity(entity.CREEPER, x, 1, z)
    elif type=='spider':
        mc.spawnEntity(entity.SPIDER, x, 1, z)

7 23 zombie
1 22 zombie
12 13 skeleton
3 30 skeleton
22 32 spider
24 30 spider
58 78 skeleton
50 79 skeleton
58 70 skeleton
60 69 skeleton
57 60 skeleton
58 52 skeleton
58 31 skeleton
57 4 skeleton
47 5 skeleton
2 58 creeper
3 51 creeper
14 76 creeper
42 75 enderman
51 49 enderman
50 84 enderman
31 32 spider
33 31 spider
49 30 spider
49 40 spider
50 22 spider
31 11 spider
10 77 zombie
6 77 zombie
10 75 zombie
7 75 zombie
3 72 zombie
6 72 zombie
4 72 zombie
2 74 zombie
5 74 zombie
2 74 creeper
2 77 creeper
7 87 skeleton
12 22 skeleton
21 41 skeleton
23 31 skeleton
20 31 skeleton
3 32 skeleton


In [24]:
# gamerule doMobSpawning false
# spawnpoint
mc.player.setPos(5,1,5)

In [115]:
mc.spawnEntity(entity.ZOMBIE,5,0,5)

2444